# Resource Estimation for simulating a 2D Heisenberg Model Hamiltonian

In this Python + Q# notebook we demonstrate how to *efficienty* estimate the resources for simulating a Heisenberg model Hamiltonian on an $N \times N$ 2D
lattice using a *fourth-order Trotter Suzuki product formula* assuming a 2D planar qubit architecture with nearest-neighbor connectivity.

First, we load the necessary packages.

In [ ]:
import qsharp
import pandas as pd

## Background: 2D Heisenberg Model

The quantum Heisenberg model is a statistical mechanical model used in the study of magnetic systems. The family of Heisenberg model Hamiltonians considered in this notebook consist of three types of interaction terms between adjacent lattice sites: $\{X \otimes X, Y \otimes Y, Z \otimes Z\}$. For our purposes we consider that the interaction strength $J$ is the same for each term. Formally, the Heisenberg model Hamiltonian on an $N \times N$ lattices divided into sets of commuting terms is formulated as:
$$ H = \underbrace{J \sum_{i, j} X_i \otimes X_j}_{A} + \underbrace{J \sum_{i, j} Z_i \otimes Z_j}_{B} + \underbrace{J \sum_{i, j} Y_i \otimes Y_j}_{C}.$$

TThe time evolution $e^{-iHt}$ for the Hamiltonian is simulated with the fourth-order Trotter-Suzuki product formula so that any errors in simulation are sufficiently small. Essentially, this is done by simulating the evolution for small slices of time $\Delta$ and repeating this for `nSteps` $= \lceil t/\Delta \rceil$ to obtain the full time evolution. The Trotter-Suzuki formula for higher orders can be recursively defined using a *fractal decomposition* as discussed in Section 3 of [Hatanao and Suziki's survey](https://link.springer.com/chapter/10.1007/11526216_2). Then the fourth order formula $U_4(\Delta)$ can be constructed using the second-order one $U_2(\Delta)$ as follows.
$$
\begin{aligned}
U_2(\Delta) & = e^{-iA\Delta/2} e^{-iB\Delta/2} e^{-iC\Delta} e^{-iB\Delta/2} e^{-iA\Delta/2}; \\
U_4(\Delta) & = U_2(p\Delta)U_2(p\Delta)U_2((1 - 4p)\Delta)U_2(p\Delta)U_2(p\Delta); \\
p & = (4 - 4^{1/3})^{-1}.
\end{aligned}
$$

For the rest of the notebook, we will present the code that computes the time evolution in a step by step fashion.

## Implementation

### Helper Functions

Note that expanding $U_4(\Delta)$ to express it in terms of $A, B, C$ gives:
$$
\begin{aligned}
U_4(\Delta) & = e^{-iAp\Delta/2} \ T_1 \ T_2 \ T_1 \ e^{-iAp\Delta/2}, \\
\text{where } T_1 & = e^{-iBp\Delta/2} e^{-iCp\Delta} e^{-iBp\Delta/2} e^{-iAp\Delta} e^{-iBp\Delta/2} e^{-iCp\Delta} e^{-iBp\Delta/2}, \\
\text{and } T_2 & = e^{-iA(1 - 3p)\Delta/2} e^{-iB(1-4p)\Delta/2} e^{-iC(1-4p)\Delta} e^{-iB(1-4p)\Delta/2} e^{-iA(1 - 3p)\Delta/2}.
\end{aligned}
$$

The above equation has $21$ exponential terms and works for one time step. For `nSteps` $> 1$ time steps, some adjacent terms can be merged to give $20\lceil t/\Delta \rceil+1$ exponential terms for time evolution $e^{-iHt}$.

The function below sets the rotation angles used to apply the exponentials involving $A, B$ and $C$ in the formula above.

In [ ]:
%%qsharp
function SetAngleSequence(p : Double, dt : Double, J : Double) : Double[] {

    let len1 = 8;
    let len2 = 4;
    let valLength = 2*len1 + len2;

    mutable values = [0.0, size=valLength];

    let val1 = -J*p*dt/2.0;
    let val2 = -J*(1.0-3.0*p)*dt/2.0;
    let val3 = -J*(1.0-4.0*p)*dt/2.0;

    // Angles bookending Term2
    set values w/= len1 <- val2;
    set values w/= len1+len2 <- val2;

    for i in 0..len1-1 {
        
        if (i % 2 == 0) {
            set values w/= i <- val1*2.0;
        }
        else {
            set values w/= i <- val1;
        }
    }

    for i in len1+1..len1+len2-1 {
        
        if (i % 2 == 0) {
            set values w/= i <- val3*2.0;
        }
        else {
            set values w/= i <- val3;
        }
    }

    for i in len1+len2+1..valLength-1 {
        
        if (i % 2 == 0) {
            set values w/= i <- val1*2.0;
        }
        else {
            set values w/= i <- val1;
        }
    }

    return values;
}

### Quantum operations

There are three kinds of operations applied to neighboring pairs of qubits on the lattice: (i) $e^{-i(X \otimes X)\theta}$; (ii) $e^{-i(Z \otimes Z)\theta}$; and (iii) $e^{-i(Y \otimes Y)\theta}$.

The following operation applies $e^{i(P \otimes P)\theta}$ for Pauli $P \in \{X, Y, Z\}$ on pairs of horizontally (resp. vertically) neighboring qubits in a single row of the lattice if `dir` is `True` (resp. `False`). Additionally, the exponential is applied only to pairs where the first qubit id is even (resp. odd) if `grp` is `True` (resp. `False`).

In [ ]:
%%qsharp
operation ApplyPoPRot(n : Int, m : Int, qArr : Qubit[][], P : Pauli, theta : Double, dir : Bool, grp : Bool) : Unit {

    let start = (grp ? 0 | 1);    // Choose either odd or even indices based on group number
    let P_op = [P, P];
    let jmp = 2;
    let end = dir ? m-2 | m-1;
    let v_end = dir? n-1 | n-2;

    for row in 0..v_end {
        for col in start..jmp..end {    // Either odd or even columns are picked based on group number
            
            let row2 = dir? row | row+1;    // Choose the next row if for vertical direction 
            let col2 = dir? col+1 | col;    // Choose the next column for horizontal direction

            Exp(P_op, theta, [qArr[row][col], qArr[row2][col2]]);
        }
    }
}

The next operation puts everything together and calls the operations needed to
simulate the Heisenberg model Hamiltonian using a fourth order product formula.
Observe that the `ApplyPoPRot` operation is called four times for each row for different
choices of direction and starting index to ensure all possible pairs of qubits
are appropriately considered.

The various parameters taken in by the operation correspond to:

- `J`: parameter by which the Hamiltonian terms are scaled.
- `N1`, `N2`: row and columns size of the lattice.
- `totTime`: the total time for the Trotter simulation.
- `dt` : the step size for the simulation, sometimes denoted as $\Delta$.

In [ ]:
%%qsharp
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;

operation HeisModel2DSim(N1 : Int, N2 : Int, J : Double, totTime : Double, dt : Double) : Unit {

    use qs = Qubit[N1*N2];

    let qubitArray = Chunks(N2, qs); // Create a 2D array of qubits

    let len = Length(qs);

    let p = 1.0/(4.0 - 4.0^(1.0/3.0));

    let nsteps = Ceiling(totTime/dt);

    let seqLen = 20*nsteps+1;

    let angSeq = SetAngleSequence(p, dt, J);

    let paulis = [PauliX, PauliZ, PauliY];

    for i in 0..seqLen-1 {
        
        let j = (i%4==0) ? 0 | ( (i%2==1) ? 1 | 2); // Choose which pauli rotation will be applied
        let theta = (i==0 or i==seqLen-1) ? -J*p*dt/2.0 | angSeq[i%20]; 

        for (dir, grp) in [(true, true), (true, false), (false, true), (false, false)] {
            ApplyPoPRot(N1, N2, qubitArray, paulis[j], theta, dir, grp);
        }
    }

    //Resetting qubits before release;
    ResetAll(qs);
}

Note that for typical choices of `totTime` and `dt` used in real-world applications, the `seqLen` can range in the hundreds or thousands of terms each of whose resources will be individually estimated. This results in a lot of time consumed (> 15 min) to compute the resource estimates. However, from the expansion of $U_4(\Delta)$, it is clear that there are a few unique terms that repeat `nsteps` number of times. Hence, to efficiently estimate resources, we compute the resources for running one step of $U_4(\Delta)$ and multiply the cost by `nsteps` repetitions.

In [ ]:
%%qsharp
open Microsoft.Quantum.ResourceEstimation;

operation EstimateHeisModel2DSim(N1 : Int, N2 : Int, J : Double, totTime : Double, dt : Double) : Unit {

    use qs = Qubit[N1*N2];

    let qubitArray = Chunks(N2, qs); // Create a 2D array of qubits

    let len = Length(qs);

    let p = 1.0/(4.0 - 4.0^(1.0/3.0));

    let t = Ceiling(totTime/dt);

    let angSeq = SetAngleSequence(p, dt, J);

    let paulis = [PauliX, PauliZ, PauliY];

    within {
        RepeatEstimates(t);
    }
    apply {
        for i in 0..19 {
            let j = (i%4==0) ? 0 | ( (i%2==1) ? 1 | 2); // Choose which pauli rotation will be applied
            let theta = angSeq[i%20]; 

            for (dir, grp) in [(true, true), (true, false), (false, true), (false, false)] {
                ApplyPoPRot(N1, N2, qubitArray, paulis[j], theta, dir, grp);
            }
        }
    }

    let angle = -J*p*dt/2.0;
    for (dir, grp) in [(true, true), (true, false), (false, true), (false, false)] {
        ApplyPoPRot(N1, N2, qubitArray, paulis[0], angle, dir, grp);
    }

    //Resetting qubits before release;
    ResetAll(qs);
}

## Getting logical resource counts

For the purpose of generating the rQOPS for some target runtime, it suffices that we obtain the logical resource estimates to simulate the Heisenberg model Hamiltonian. We consider three problem instances with lattice sizes $\{20 \times 20, 30 \times 30, 40 \times 40\}$ with $J = 1.0$. These instances are simulated for a total time of $1000$ s, with step size `dt`$ = 0.1$, and overall probability of failure $\varepsilon = 0.01$. Any one of the six pre-defined qubit parameters will do to obtain the logical coounts and in this notebook we choose a Majorana based qubit with the `floquet code`.

In [ ]:
# Set up problem parameters
N1 = [20, 30, 40]
N2 = [20, 30, 40]
J = 1.0
totTime = 1000.0
dt = 0.1

We submit a resource estimation job with all the problem instances sequentially and collect the estimates in `results`.

In [ ]:
# Submit jobs
results = []

for i in range(3):
    qsharp_string = f"EstimateHeisModel2DSim({N1[i]}, {N2[i]}, {J}, {totTime}, {dt})"

    result = qsharp.estimate(qsharp_string, params={"errorBudget": 0.01, "qubitParams": {"name": "qubit_maj_ns_e6"}, "qecScheme": {"name": "floquet_code"}, "constraints": {"logicalDepthFactor": 4}})
    results.append(result)

To see the complete information provided when invoking the resource estimator, we output the result for the $20 \times 20$ lattice by displaying `results[0]`

In [ ]:
# Displaying estimates for 20x20 lattice size.
results[0]
# Change index to 1 (resp. 2) for 30x30 (resp. 40x40) lattice size.

## Visualizing and understanding the results

### Result summary table

In [ ]:
# Define function to display information in summary format
def get_summary_table(results, labels):
    logical_qubits = []
    logical_depth = []
    t_states = []
    code_distance = []
    t_factories = []
    t_factory_fraction = []
    physical_qubits = []
    rqops = []
    runtime = []
    logical_error = []

    for i in range(3): 
        logical_qubits.append(results[i]['physicalCounts']['breakdown']['algorithmicLogicalQubits'])
        logical_depth.append(results[i]['physicalCountsFormatted']['logicalDepth'])
        t_states.append(results[i]['physicalCountsFormatted']['numTstates'])
        t_factories.append(results[i]['physicalCounts']['breakdown']['numTfactories'])
        logical_error.append(results[i]['physicalCountsFormatted']['requiredLogicalQubitErrorRate'])
        physical_qubits.append(results[i]['physicalCountsFormatted']['physicalQubits'])
        rqops.append(results[i]['physicalCountsFormatted']['rqops'])
        runtime.append(results[i]['physicalCountsFormatted']['runtime'])
        code_distance.append(results[i]['logicalQubit']['codeDistance'])
        t_factory_fraction.append(results[i]['physicalCountsFormatted']['physicalQubitsForTfactoriesPercentage'])

    data = pd.DataFrame()
    pd.options.display.float_format = '{:.2E}'.format
    data['Logical qubits'] = logical_qubits
    data['Logical depth'] = logical_depth
    data['Logical error'] = logical_error
    data['T states'] = t_states
    # data['T states'] = data['T states'].astype('float64')
    data['Code Distance'] = code_distance
    data['T factories'] = t_factories
    data['T factory fraction'] = t_factory_fraction
    data['Physical qubits'] = physical_qubits
    data['rQOPS'] = rqops
    data['Physical runtime'] = runtime
    data.index = labels

    return data

# Display summarized information for all problem instances
labels = ["Hei20", "Hei30", "Hei40"]
table = get_summary_table(results, labels)
table

> Note that in general, there is a trade-off between the logical depth and number of T factories used. 

> To ensure that T factories do not dominate the resource requirements, we set the `logical_depth_factor`${}=4$ adding some number of `noops` to increase the logical depth.

### Getting the target rQOPS

While the resource estimator generates a runtime for the given hardware profile, we want to set a target runtime of 1 week i.e., 604800 seconds to obtain a practical quantum advantage. We collect the previous results to compute the corresponding target rQOPS as 
$$ \text{Target rQOPS} = \frac{\text{Logical qubits}\cdot\text{Logical Depth}}{\text{Target runtime}}$$

In [ ]:
def get_target_rqops(results, labels):

    target_runtime = 604800
    logical_qubits = []
    logical_depth = []
    target_rqops = []
    logical_error = []

    for i in range(3): 
        logical_qubits.append(results[i]['physicalCounts']['breakdown']['algorithmicLogicalQubits'])
        logical_depth.append(results[i]['physicalCountsFormatted']['logicalDepth'])
        logical_error.append(results[i]['physicalCountsFormatted']['requiredLogicalQubitErrorRate'])
        target_rqops.append(round(results[i]['physicalCounts']['breakdown']['algorithmicLogicalQubits'] * results[i]['physicalCounts']['breakdown']['logicalDepth'] / target_runtime))

    data = pd.DataFrame()
    pd.options.display.float_format = '{:.2e}'.format
    data['Logical qubits'] = logical_qubits
    data['Logical depth'] = logical_depth
    data['Logical error'] = logical_error
    data['Target rQOPS'] = target_rqops
    data['Target rQOPS'] = data['Target rQOPS'].astype('float64')
    data.index = labels

    return data

rQOPS_table = get_target_rqops(results, labels)
rQOPS_table

## Next steps

Feel free to use this notebook as a starting point for your own experiments.  For
example, you can

* explore how the results change considering other problem instances of the Heisenberg model
* explore space- and time-trade-offs by changing the value for
  `logical_depth_factor` or `max_t_factories`
* visualize these trade-offs with the space and time diagrams
* use other or customized qubit parameters